# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H. Jiang  ->  H. Jiang  |  ['H. Jiang']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
F. Zagaria  ->  F. Zagaria  |  ['F. Zagaria']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Arxiv has 79 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2601.15926
extracting tarball to tmp_2601.15926... done.
Retrieving document from  https://arxiv.org/e-print/2601.15948


extracting tarball to tmp_2601.15948... done.


M. Benisty  ->  M. Benisty  |  ['M. Benisty']
F. Zagaria  ->  F. Zagaria  |  ['F. Zagaria']


Issues with the citations
repeated bibliography entry: Szulgyi2020
Retrieving document from  https://arxiv.org/e-print/2601.15969
extracting tarball to tmp_2601.15969...

 done.
Retrieving document from  https://arxiv.org/e-print/2601.16048


extracting tarball to tmp_2601.16048... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.15948-b31b1b.svg)](https://arxiv.org/abs/2601.15948) | **A 2 au resolution view by ALMA of the planet-hosting WISPIT 2 disk**  |
|| S. Facchini, et al. -- incl., <mark>M. Benisty</mark>, <mark>F. Zagaria</mark> |
|*Appeared on*| *2026-01-23*|
|*Comments*| *Accepted for publication on ApJL; 14 pages, 10 figures*|
|**Abstract**|            We present deep, high spatial resolution interferometric observations of 0.88 mm continuum emission from the TYC 5709-354-1 system, hereafter WISPIT 2, obtained with the goal of detecting circumplanetary emission in the vicinity of the newly discovered WISPIT 2b planet. Observations with the most extended baseline configuration offered by ALMA, achieving an angular resolution of $25 \times 17$ mas ($3.3 \times 2.2$ au), revealed a single, narrow ring with a deprojected radius of 144.4 au and width of 7.2 au, and no evidence of circumplanetary emission within the cavity. Injection and recovery tests demonstrate that these observations can rule out point-like emission at the location of WISPIT 2b brighter than $\approx 45$ $\mu$Jy at the $3\sigma$ level. While these data can rule out PDS 70c like circumplanetary emission, the upper limit is consistent with empirical mass-flux relationships extrapolated from the stellar regime. Visibility modeling of the continuum ring confirms that WISPIT 2b lies significantly interior to the mm dust ring, raising doubts about the ability of WISPIT 2b to be the only driver of the dust structure. Possible solutions include either another lower mass companion, residing between WISPIT 2b and the cavity edge, likely in the gap seen by SPHERE at $\sim130$ au, or that WISPIT 2b is either substantially more massive than IR-photometry based estimates ($\sim15$ $M_{\rm Jup}$) or on a moderately eccentric orbit. The combination of observations sensitive to the gas and dust distributions on larger spatial scales and dedicated hydrodynamical modeling will help differentiate between scenarios.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.15926-b31b1b.svg)](https://arxiv.org/abs/2601.15926) | **Out-of-Sample Validation of MagNet**  |
|| A. Yesmanchyk, et al. -- incl., <mark>H. Jiang</mark> |
|*Appeared on*| *2026-01-23*|
|*Comments*| **|
|**Abstract**|            Machine learning is starting to be used in almost every industry and academic research, and solar physics is no exception. A newly developed machine learning model named MagNet helps us to tackle some of the most serious challenges in data mining by generating transverse fields of solar active regions. Being trained on line-of-sight magnetograms from Michelson Doppler Imager at Solar and Heliospheric Observatory (SOHO/MDI), H{\alpha} maps from Big Bear Solar Observatory and Kanzelhohe Solar Observatory and vector magnetograms from Helioseismic and Magnetic Imager at Solar Dynamic Observatory (SDO/HMI), this model provides vector magnetograms in active regions for SOHO/MDI data covering the strong solar cycle 23. In this study, we performed out-of-sample validation of the MagNet model with data from Imaging Vector Magnetograph (IVM) at Mees Solar Observatory, which was not included in the training process. Our results show good correlation between the AI generated data and the observed vector magnetograms and therefore strengthen the confidence of implementing MagNet to the entire SOHO/MDI archive and future scientific analysis of the AI generated data.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.16048-b31b1b.svg)](https://arxiv.org/abs/2601.16048) | **Unveiling the Spectral Morphological Division of Fast Radio Bursts with CHIME/FRB Catalog 2**  |
|| W.-P. Sun, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2026-01-23*|
|*Comments*| *13 pages, 6 figures*|
|**Abstract**|            Fast radio bursts (FRBs) are commonly divided into repeating and apparently non-repeating sources, but whether these represent distinct physical populations remains uncertain. In this work, we apply an unsupervised machine learning methods combining Uniform Manifold Approximation and Projection (UMAP) with density-based clustering to analyze CHIME/FRB Catalog 2. We find that FRBs remain primarily separated into two clusters in the multi-dimensional parameter space, with a recall of 0.94 for known repeaters, indicating strong robustness. Consistent with Catalog 1 analyses, we confirm that the spectral morphology parameter, specifically spectral running remains the key discriminator between the two populations, indicating that narrowband emission is an intrinsic and persistent property of repeating FRBs. With the enlarged Catalog 2 sample, we further identify a stable subclass of atypical repeaters (about $6\%$ of repeating bursts) that are broadband, shorter in duration, and more luminous, resembling non-repeating bursts. The Nonrepeater-like cluster also shows higher inferred energies and dispersion measures, consistent with a scenario in which apparently non-repeating FRBs may result from observational incompleteness, with low-energy repeating bursts remaining undetected. Our results provide new statistical evidence for a physical connection between repeating and non-repeating FRBs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.15969-b31b1b.svg)](https://arxiv.org/abs/2601.15969) | **Gaia20fnr: A binary-lens microlensing event with full orbital motion revealed by four space telescopes**  |
|| M. Wicker, et al. |
|*Appeared on*| *2026-01-23*|
|*Comments*| *22 pages, 11 figures, submitted to A&A on 22.01.2026*|
|**Abstract**|            The microlensing event Gaia20fnr is a long-duration, non-caustic-crossing binary-lens event at high Galactic latitude. Triggered by a photometric rise detected by the Gaia space mission, the event was followed up with observations from multiple ground-based facilities and four space telescopes: Gaia, NEOWISE, Swift, and TESS. We characterize the Gaia20fnr microlensing system by determining the physical and orbital properties of the binary lens, the nature of the luminous source, and the kinematics of both the source and the lens. We employed a binary-lens microlensing model including full Keplerian orbital motion and annual microlens parallax to fit the photometric data. The event is best explained by a K2 giant source at $D_{\rm S} = 3.10 \pm 0.10\,\mathrm{kpc}$ lensed by a stellar binary composed of $M_{\rm L,1} = 0.46 \pm 0.06\,M_\odot$ and $M_{\rm L,2} = 0.52 \pm 0.06\,M_\odot$ at a distance of $D_{\rm L} = 0.54 \pm 0.05\,\mathrm{kpc}$. The light curve exhibits strong signatures of orbital motion and requires a full Keplerian model with a period of $P = 0.67 \pm 0.04\,\mathrm{yr}$ and a radial-velocity semi-amplitude of $K_1 = 16.9 \pm 0.9\,\mathrm{km\,s^{-1}}$. Gaia20fnr is one of the few microlensing events for which a complete Keplerian binary-lens solution has been derived. The model can be tested with follow-up radial-velocity and high-resolution imaging observations as well as forthcoming Gaia DR4 and DR5 astrometric time-series data. Its long duration, multi-peak structure, and extensive coverage make it a benchmark for studying faint nearby low-mass binaries through microlensing.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2601.15948/./figures/gapwidth_q.png', 'tmp_2601.15948/./figures/recovered_CPD_flux_v2.png', 'tmp_2601.15948/./figures/WISPIT_fig1_combined.png']
copying  tmp_2601.15948/./figures/gapwidth_q.png to _build/html/
copying  tmp_2601.15948/./figures/recovered_CPD_flux_v2.png to _build/html/
copying  tmp_2601.15948/./figures/WISPIT_fig1_combined.png to _build/html/
exported in  _build/html/2601.15948.md
    + _build/html/tmp_2601.15948/./figures/gapwidth_q.png
    + _build/html/tmp_2601.15948/./figures/recovered_CPD_flux_v2.png
    + _build/html/tmp_2601.15948/./figures/WISPIT_fig1_combined.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{km~s^{-1} }$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# A 2 au resolution view by ALMA of the planet-hosting WISPIT 2 disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2601.15948-b31b1b.svg)](https://arxiv.org/abs/2601.15948)<mark>Appeared on: 2026-01-23</mark> -  _Accepted for publication on ApJL; 14 pages, 10 figures_

</div>
<div id="authors">

F. Stefano, et al. -- incl., <mark>M. Benisty</mark>, <mark>F. Zagaria</mark>

</div>
<div id="abstract">

**Abstract:** We present deep, high spatial resolution interferometric observations of 0.88 mm continuum emission from the TYC 5709-354-1 system, hereafter WISPIT 2, obtained with the goal of detecting circumplanetary emission in the vicinity of the newly discovered WISPIT 2b planet. Observations with the most extended baseline configuration offered by ALMA, achieving an angular resolution of $25 \times 17$ mas ( $3.3\times2.2$ au), revealed a single, narrow ring with a deprojected radius of $144.4$ au and width of $7.2$ au, and no evidence of circumplanetary emission within the cavity. Injection and recovery tests demonstrate that these observations can rule out point-like emission at the location of WISPIT 2b brighter than ${\approx} 45 \mu$ Jy at the $3\sigma$ level. While these data can rule out PDS 70c like circumplanetary emission, the upper limit is consistent with empirical mass-flux relationships extrapolated from the stellar regime. Visibility modeling of the continuum ring confirms that WISPIT 2b lies significantly interior to the mm dust ring, raising doubts about the ability of WISPIT 2b to be the only driver of the dust structure. Possible solutions include either another lower mass companion, residing between WISPIT 2b and the cavity edge, likely in the gap seen by SPHERE at $\sim130 $ au, or that WISPIT 2b is either substantially more massive than IR-photometry based estimates ( ${\sim} 15 M_{\rm Jup}$ ) or on a moderately eccentric orbit. The combination of observations sensitive to the gas and dust distributions on larger spatial scales and dedicated hydrodynamical modeling will help differentiate between scenarios.

</div>

<div id="div_fig1">

<img src="tmp_2601.15948/./figures/gapwidth_q.png" alt="Fig3" width="100%"/>

**Figure 3. -** Gap width (in units of the gap radius) as a function of the planet-to-star mass ratio. For systems hosting detected protoplanets, $R_{\rm gap} = a_{\rm p}$, where $a_{\rm p}$ is the orbital radius of the planet. The candidate protoplanets in (some of) the gaps in the DSHARP sample \citep{Andrews_ea_2018,Huang2018} and the Taurus survey of \citet{Long2018} are displayed as gray dots, with putative planet masses estimated by \citet{Ruzza2025} and stellar masses by \citet{Andrews_ea_2018,Long2019}. PDS 70b and c are over-plotted in blue and WISPIT 2b in yellow. The dashed and dashed dotted black lines are the expected gap width to planet mass trends predicted by \citet{Rosotti2016} and \citet{Facchini2018}, their extrapolation to the whole planet mass range we considered is shown in gray. (*fig:R_mm_a_planet*)

</div>
<div id="div_fig2">

<img src="tmp_2601.15948/./figures/recovered_CPD_flux_v2.png" alt="Fig1" width="100%"/>

**Figure 1. -** Injected ($F_{\rm CPD,inj}$) vs recovered ($F_{\rm CPD,rec}$) flux density of a point-source at the location of WISPIT 2b. The recovered flux density is estimated as the peak intensity within 1.5 beams at that location. The ribbon indicates the $1\sigma$ uncertainty on the peak intensity estimate. (*fig:CPD_flux_recovery*)

</div>
<div id="div_fig3">

<img src="tmp_2601.15948/./figures/WISPIT_fig1_combined.png" alt="Fig7" width="100%"/>

**Figure 7. -** 0.88 mm continuum intensity of the WISPIT 2 system, showing a thin ring at $1$\farcs$086$($144.4 $au deprojected distance). The right panel shows the same image, with white dashed lines indicating the radial peaks observed in IR scattered light by \citet{van_Capelleveen2025}, projected onto the disk midplane. The numbering ($R_{N,{\rm S}}$) follows the one by \citet{van_Capelleveen2025}. The location of the WISPIT 2b planet \citep{van_Capelleveen2025,Close2025} is highlighted in the right panel. The ALMA resolution element is shown in the bottom left of each panel. (*fig:WISPIT_2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2601.15948"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

112  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
